In [76]:
import pandas as pd
import seaborn as sns
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [9]:
data = pd.read_csv("ObesityDataSet.csv")

In [10]:
data

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


Catboost

In [13]:
cat_train, cat_test = train_test_split(data,
                test_size=0.2,
                train_size=0.8,
                random_state=1,shuffle=True,
                stratify=data["Gender"])

cat_train_y = cat_train["NObeyesdad"]
cat_train_x = cat_train.drop(columns = ["NObeyesdad"])
cat_test_y = cat_test["NObeyesdad"]
cat_test_x = cat_test.drop(columns = ["NObeyesdad"])

In [14]:
cat_train_pool = cb.Pool(cat_train_x, 
                  cat_train_y, 
                  cat_features=[0,4,5,8,9,11,14,15])

cat_test_pool = cb.Pool(cat_test_x, 
                  cat_test_y, 
                  cat_features=[0,4,5,8,9,11,14,15])

In [33]:
model = cb.CatBoostClassifier(iterations=2000, 
                          depth=3, 
                          learning_rate=0.1, 
                          loss_function='MultiClass')

In [ ]:
model.fit(cat_train_pool)
preds = model.predict(cat_test_pool)

In [35]:
balanced_accuracy_score(cat_test_y, preds)

0.9673505123162727

Классификация с использованием KNN

In [52]:
d_data = pd.get_dummies(data = data.drop(columns=["NObeyesdad"]), drop_first=True)

In [54]:
d_data["NObeyesdad"] = data["NObeyesdad"] 

In [60]:
train, test = train_test_split(d_data,
                test_size=0.2,
                train_size=0.8,
                random_state=1,shuffle=True,
                stratify=data["Gender"])

In [61]:
train_y = train["NObeyesdad"]
train_x = train.drop(columns = ["NObeyesdad"])
test_y = test["NObeyesdad"]
test_x = test.drop(columns = ["NObeyesdad"])

In [80]:
knn = KNeighborsClassifier(n_neighbors = 3).fit(train_x, pd.get_dummies(data = train_y, drop_first=True))
print(knn.score(test_x, pd.get_dummies(data = test_y, drop_first=True)))

0.8888888888888888


Наивный баесовский классификатор

In [84]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(train_x, train_y)
print(gnb.score(test_x, test_y))

0.5555555555555556
